In [0]:
from pyspark.sql import SparkSession

In [0]:
sparkSession = SparkSession.builder.appName('Spark Kafka Consumer Örnek').getOrCreate()

In [0]:
sparkSession

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
rawDF = sparkSession.readStream.format("kafka").option("kafka.bootstrap.servers","159.223.188.228:9092").option("startingOffsets","earliest").option("subscribe","person").load()

In [0]:
rawDF.show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-1353366328657068> in <module>
----> 1 rawDF.show()

/databricks/spark/python/pyspark/sql/dataframe.py in show(self, n, truncate, vertical)
    488         """
    489         if isinstance(truncate, bool) and truncate:
--> 490             print(self._jdf.showString(n, 20, vertical))
    491         else:
    492             print(self._jdf.showString(n, int(truncate), vertical))

/databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303         answer = self.gateway_client.send_command(command)
-> 1304         return_value = get_return_value(
   1305             answer, self.gateway_client, self.target_id, self.name)
   1306 

/databricks/spark/python/pyspark/sql/utils.py in deco(*a, **kw)
    121                 # Hide where the exception came from that shows a non-Pythonic

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
schema = StructType() \
         .add("name",StringType()) \
         .add("surname",StringType()) \
         .add("age",IntegerType()) \
         .add("city",StringType())

In [0]:
parsedDF = rawDF.select(from_json(col("value").cast("string"),schema).alias("kafkadata")).select("kafkadata.*")

In [0]:
parsedDF.show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-1353366328657072> in <module>
----> 1 parsedDF.show()

/databricks/spark/python/pyspark/sql/dataframe.py in show(self, n, truncate, vertical)
    488         """
    489         if isinstance(truncate, bool) and truncate:
--> 490             print(self._jdf.showString(n, 20, vertical))
    491         else:
    492             print(self._jdf.showString(n, int(truncate), vertical))

/databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303         answer = self.gateway_client.send_command(command)
-> 1304         return_value = get_return_value(
   1305             answer, self.gateway_client, self.target_id, self.name)
   1306 

/databricks/spark/python/pyspark/sql/utils.py in deco(*a, **kw)
    121                 # Hide where the exception came from that shows a non-Pytho

In [0]:
resultDF = parsedDF.filter("age>25")

In [0]:
parsedDF.createOrReplaceTempView("person")

In [0]:
%sql
SELECT * FROM person WHERE age>25

name,surname,age,city
Yasin,Kaya,32,Tekirdağ


In [0]:
resultDF.selectExpr("CAST(name AS STRING) AS key", "to_json(struct(*)) AS value").writeStream.format("kafka").option("kafka.bootstrap.servers","159.223.188.228:9092").option("checkpointLocation","/point").option("topic","sonuc2").start()

Out[47]: <pyspark.sql.streaming.StreamingQuery at 0x7feacef3adf0>